# Tutorial CS002: Load Forecasting with Transformer

This tutorial demonstrates how to use a Transformer-based architecture for short-term load forecasting. You will learn how to:
1. Handle time series as sequences for self-attention.
2. Implement a Transformer encoder model.
3. Compare Transformer performance with baseline models.

In [ ]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

sys.path.append(os.path.abspath('../../src'))
from models.transformer import TransformerModel
from preprocessing.pipeline import DataPipeline
from utils.metrics import evaluate_all
from utils.plotting import plot_forecast, set_style
from utils.trainer import Trainer
from data.download import download_public_dataset

set_style()

In [ ]:
data_path = download_public_dataset('D002', save_dir='../../data/raw')
df = pd.read_csv(data_path)
pipeline = DataPipeline()
train_df, val_df, test_df = pipeline.split_data(df)

target_col = 'load'
train_scaled = pipeline.fit_transform(train_df, [target_col])
val_scaled = pipeline.transform(val_df, [target_col])
test_scaled = pipeline.transform(test_df, [target_col])

WINDOW_SIZE = 48  # Transformers benefit from longer contexts
X_train, y_train = pipeline.create_sequences(train_scaled, WINDOW_SIZE)
X_val, y_val = pipeline.create_sequences(val_scaled, WINDOW_SIZE)
X_test, y_test = pipeline.create_sequences(test_scaled, WINDOW_SIZE)

# Transformer expects (seq_len, batch_size, input_size)
def prepare_transformer_data(X):
    return torch.from_numpy(X).float().permute(1, 0, 2)

train_loader = DataLoader(TensorDataset(prepare_transformer_data(X_train), torch.from_numpy(y_train).float()), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(prepare_transformer_data(X_val), torch.from_numpy(y_val).float()), batch_size=32)
test_loader = DataLoader(TensorDataset(prepare_transformer_data(X_test), torch.from_numpy(y_test).float()), batch_size=32)

In [ ]:
model = TransformerModel(input_size=1, d_model=32, nhead=4, num_encoder_layers=2, dim_feedforward=64, output_size=1)
trainer = Trainer(model, torch.nn.MSELoss())
trainer.train(train_loader, val_loader, epochs=5, save_path='transformer_best.pth')

In [ ]:
y_pred_scaled = trainer.predict(test_loader)
y_pred = pipeline.scaler.inverse_transform(y_pred_scaled)
y_true = pipeline.scaler.inverse_transform(y_test)

plot_forecast(y_true[:168], y_pred[:168], title="Transformer Load Forecast", save_path='../../figures/cs002_transformer_forecast.pdf')